--- Day 4: Giant Squid ---
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0

 8  2 23  4 24

21  9 14 16  7

 6 10  3 18  5

 1 12 20 15 19


 3 15  0  2 22

 9 18 13 17  5

19  8  7 25 23

20 11 10 24  4

14 21 16 12  6


14 21 17 24  4

10 16 15  9 19

18  8 23 26 20

22 11 13  6  5

 2  0 12  3  7


After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):


22 13 17 11  0         3 15  0  2 22        14 21 17 24  4

 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19

21  9 14 16  7        19  8  7 25 23        18  8 23 26 20

 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5

 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7

After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

22 13 17 11  0         3 15  0  2 22        14 21 17 24  4

 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19

21  9 14 16  7        19  8  7 25 23        18  8 23 26 20

 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5

 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7

Finally, 24 is drawn:

22 13 17 11  0         3 15  0  2 22        14 21 17 24  4

 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19

21  9 14 16  7        19  8  7 25 23        18  8 23 26 20

 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5

 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

current issue: arr is a list of numbers, but it needs to be a list of lists of numbers

In [98]:
with open('day4_puzzle_input.txt') as fp:
    string = fp.readline().strip()
    tmp_string = ''
    winning_numbers = []    # list to hold winning values in drawn order
    for i in range(len(string)):
        if ord(string[i]) != 44:
            tmp_string = tmp_string + string[i]
        else:
            winning_numbers.append(int(tmp_string))
            tmp_string = ''
    string = fp.readline().strip()
    arr = []
    if string == '':
        string = fp.readline().strip()
    while string:
        tmp_string = ''
        for i in range(len(string)):
            if ord(string[i]) != 32:
                tmp_string = tmp_string + string[i]
            else:
                if tmp_string != '':
                    arr.append(int(tmp_string))
                    tmp_string = ''
        string = fp.readline().strip()
        if string == '':
            string = fp.readline().strip()

fastest_board = []  # stores how many turns it takes each bingo board to get a bingo
bingo_board = []
x = 0   # variable for building bingo boards appropiately
while True:
    turns = 0   # How many turns each board takes to get a bingp
    x+=5
    print(f"arr = {arr}")
    bingo_board = arr[x-5:x]
    print(f"bingo_board = {bingo_board}")
    tmp_list = []
    for i in range(5):
        for j in range(5):
            tmp_list.append(bingo_board[j][i])
        bingo_board.append(tmp_list)
        tmp_list = []
    
    bingo = 0
    num = 0
    while True:
        bingo = winning_numbers[num]
        for i in range(10):
            for j in range(5):
                if bingo in bingo_board[i][j]:
                    bingo_board[i][j] = 0
        break
    
    break


    

winning_numbers = [31, 50, 79, 59, 39, 53, 58, 95, 92, 55, 40, 97, 81, 22, 69, 26, 6, 23, 3, 29, 83, 48, 18, 75, 47, 49, 62, 45, 35, 34, 1, 88, 54, 16, 56, 77, 28, 94, 52, 15, 0, 87, 93, 90, 60, 67, 68, 85, 80, 51, 20, 96, 61, 66, 63, 91, 8, 99, 70, 13, 71, 17, 7, 38, 44, 43, 5, 25, 72, 2, 57, 33, 82, 78, 89, 21, 30, 11, 73, 84, 4, 46, 14, 19, 12, 10, 42, 32, 64, 98, 9, 74, 86, 27, 24, 65, 37, 41, 76]
arr = [31, 5, 70, 8, 38, 63, 14, 91, 22, 67, 17, 47, 93, 52, 69, 29, 33, 66, 64, 19, 35, 63, 17, 48, 25, 58, 33, 14, 32, 87, 90, 66, 16, 4, 98, 72, 19, 74, 39, 29, 40, 29, 44, 17, 56, 98, 83, 62, 25, 91, 20, 60, 42, 66, 34, 77, 16, 8, 6, 50, 13, 6, 58, 39, 3, 15, 69, 5, 81, 65, 0, 85, 72, 7, 60, 42, 45, 64, 47, 53, 94, 21, 67, 0, 2, 75, 77, 15, 38, 25, 49, 99, 76, 35, 69, 4, 42, 96, 86, 84, 46, 7, 74, 65, 99, 12, 4, 38, 30, 90, 78, 94, 22, 15, 72, 52, 11, 67, 59, 3, 18, 17, 63, 53, 16, 76, 55, 5, 33, 82, 60, 51, 29, 99, 87, 95, 88, 15, 75, 61, 10, 24, 79, 28, 64, 43, 21, 48, 45, 85, 80, 7

TypeError: 'int' object is not subscriptable